# This notebook ... TODO

In [4]:
import pathlib
import yaml
import subprocess

import scanpy as sc
import anndata as ad


## Load config

In [2]:
# Get the root directory of the analysis repository
REPO_ROOT = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"], capture_output=True, text=True
).stdout.strip()

CONFIG_FILE = pathlib.Path(REPO_ROOT) / 'config.yml'
assert CONFIG_FILE.exists(), f"Config file not found at {CONFIG_FILE}"

with open(CONFIG_FILE, 'r') as file:
    config_dict = yaml.safe_load(file)

## Retrieve Path to Processed Single-Cell RNA-seq Data and relevant Metadata

In [5]:
STUDY_GEO_ID = 'GSE154600' # TODO consider whether to move this into config.yml as well
SC_DATA_PATH = pathlib.Path(config_dict['data_path']['sc_data_path'])

SC_ADATA_PATH = SC_DATA_PATH / f'{STUDY_GEO_ID}_processed'
assert SC_ADATA_PATH.exists(), f"Processed Single-cell Data path {SC_ADATA_PATH} does not exist"
SC_ADATA_FILE = SC_ADATA_PATH / f'{STUDY_GEO_ID}_processed.h5ad'
assert SC_ADATA_FILE.exists(), f"Processed Single-cell Data file {SC_ADATA_FILE} does not exist"

SC_METADATA_PATH = SC_DATA_PATH / f'{STUDY_GEO_ID}_metadata'
assert SC_METADATA_PATH.exists(), f"Single-cell Metadata path {SC_METADATA_PATH} does not exist"

## Preprocessing Before Moving to Pseudobulk

### Load and Preprocess Anndata

In [6]:
adata = sc.read_h5ad(SC_ADATA_FILE)
adata.var_names_make_unique()

Print some basic information

In [ ]:
print(adata.shape)
print(adata.var.head())
print(adata.obs.head())

(36111, 24520)
              gene_ids  n_cells     mt  n_cells_by_counts  mean_counts  \
gene_ids                                                                 
AL627309.1  AL627309.1       23  False                 23     0.000614   
AL669831.5  AL669831.5      629  False                629     0.017388   

            pct_dropout_by_counts  total_counts  
gene_ids                                         
AL627309.1              99.938567          23.0  
AL669831.5              98.319934         651.0  
                           GSM             Barcode  Cluster     cellType  \
AAACCTGAGCTGCCCA-1  GSM4675273  AAACCTGAGCTGCCCA-1        3  Macrophages   
AAACCTGCAAGCCCAC-1  GSM4675273  AAACCTGCAAGCCCAC-1        3  Macrophages   

                   hpca_celltype encode_celltype subtype  IMR_consensus  \
AAACCTGAGCTGCCCA-1    Macrophage     Macrophages     DIF          0.322   
AAACCTGCAAGCCCAC-1    Macrophage     Macrophages     IMR          0.362   

                    DIF_consensus